In [ ]:
# This is the modified Fukami-model code to train the neural network
import os
# Ignore Tensorflow Warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
import numpy as np
import pandas as pd

from keras.layers import Input, Dense, Conv2D, Concatenate, MaxPooling2D, UpSampling2D
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
from time import time
from datetime import date

In [ ]:
# Template for Summary file
summary = """Date: {}
Kernel: {}
Pooling: {}
Variable: {}
Train_Size: {}
Shuffle: {}
Patience: {}
Batch_Size: {}
Time: {}
Random_State_Train: {}
Random_State_Test: {}
sample_start: {}
sample_end: {}
min_val_loss: {}
Eval: {}
Augmentation sample size: {}
"""

In [ ]:
# Tool to add counter if filename already exists
def uniquify(paths):
    paths_out = []
    for path in paths:
        fname, ext = os.path.splitext(path)
        counter = 1

        while os.path.exists(path):
            path = "{}_{}{}".format(fname, counter, ext)
            counter += 1

        paths_out.append(path)
    return paths_out

In [ ]:
# Setting for Training datasets
kernel = 16
pooling = "DS" # Can be DS for direct simulation, but also Average Pooling, etc
var = "Hs"

# Get the current date
today = date.today().strftime("%Y%m%d")

# Prepare file locations for reference and input data
fname_HR = './Data/HR/{}/BaskCoast_{}_{{}}.csv'.format(var, var.upper())
fname_LR = './Data/LR/{}/Kernel_{}/{}/BaskCoast_{}_{{}}.csv'.format(pooling, kernel, var, var.upper())


# Beginning and end of the data set serial
sample_start = 24
sample_end = 8760
serial = np.arange(sample_start, sample_end+1, 1)
grid = (160,160)
dim = 1 # In case of multiple inputs like wave height AND period

# Beginning and end of test data serial
sample_start_test = 8761
sample_end_test = 17496
serial_test = np.arange(sample_start_test, sample_end_test+1, 1)

In [ ]:
# Learning parameters

# Training data might be shuffled, however, might affect temporal coherence
shuffle = False
# Create random integer to set a random state for train_test_split if shuffled
random_state_train, random_state_test = np.random.randint(1e6, size=2)

train_size = 0.8
patience = 30 # For early stopping
batch_size = 32 
n_augm = False # With or without data augmentation

In [ ]:
# Load the data in to memory

# Initialize input and labels arrays respectively
X_tot = np.zeros((len(serial), grid[0], grid[1], dim))
y_tot = np.zeros((len(serial), grid[0],grid[1], dim))

print("Loading data...")

# Iterate through the training data set and save reference and input in arrays
for i, s in enumerate(serial):

    # Load reference data
    df = pd.read_csv(fname_HR.format(s), header=None, delim_whitespace=False)
    y_tot[i,:,:,0] = df.values

    # Load input data
    df = pd.read_csv(fname_LR.format(s), header=None, delim_whitespace=False)
    X_tot[i,:,:,0] = df.values
    

# Split into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_tot, y_tot, train_size=train_size, shuffle=shuffle,
                                                  random_state=random_state_train)

# Discard to free memory
del X_tot, y_tot

# print("Performing data augmentation")
# # Data Augmentation Process
# # If number of augmentation samples is not specified, it is as big as original training set
# shape = (n_augm, grid[0], grid[1], dim)
# if not n_augm:
#     shape = X_train.shape
#
# X_augm = np.empty(shape)
# y_augm = np.empty(shape)
#
# X_train[np.random.choice(X_train.shape[0], n_augm, replace=False)]
#
# # Arbitrary offset between 0 and 5
# offset = np.random.uniform(0, 5, size=(shape[0],1,1,1)) * np.ones(shape)
#
#
# # If only a a subset is augmented, choose a random subset of X_train
# if n_augm:
#     np.random.seed(random_state_train)
#     X_augm = offset + X_train[np.random.choice(X_train.shape[0], n_augm, replace=False)]
#     np.random.seed(random_state_train)
#     y_augm = offset + y_train[np.random.choice(y_train.shape[0], n_augm, replace=False)]
# else:
#     X_augm = offset + X_train 
#     y_augm = offset + y_train
#
# X_train = np.concatenate((X_train, X_augm))
# y_train = np.concatenate((y_train, y_augm))

# Convert all NaNs to zero
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)

X_valid = np.nan_to_num(X_valid)
y_valid = np.nan_to_num(y_valid)

# Shuffle again, if necessary
# np.random.seed(random_state_train)
# np.random.shuffle(X_train)
# np.random.seed(random_state_train)
# np.random.shuffle(y_train)

print("X_train shape: ", X_train.shape)
# print("X_augm shape: ", X_augm.shape)
print("X_valid shape: ", X_valid.shape)

# del X_augm, y_augm

In [ ]:
# Initialize Downsampled Skip-Conncetion - Multi Scale model by Fukami 2019
print("Initializing Model")
# Original Model
input_img = Input(shape=(grid[0], grid[1], dim))

#Downsampled skip-connection model
down_1 = MaxPooling2D((8,8), padding='same')(input_img)
x1 = Conv2D(32, (3,3), activation='relu', padding='same')(down_1)
x1 = Conv2D(32, (3,3), activation='relu', padding='same')(x1)
x1 = UpSampling2D((2,2))(x1)

down_2 = MaxPooling2D((4,4), padding='same')(input_img)
x2 = Concatenate()([x1, down_2])
x2 = Conv2D(32, (3,3), activation='relu', padding='same')(x2)
x2 = Conv2D(32, (3,3), activation='relu', padding='same')(x2)
x2 = UpSampling2D((2,2))(x2)

down_3 = MaxPooling2D((2,2), padding='same')(input_img)
x3 = Concatenate()([x2, down_3])
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(x3)
x3 = Conv2D(32, (3,3), activation='relu', padding='same')(x3)
x3 = UpSampling2D((2,2))(x3)

x4 = Concatenate()([x3, input_img])
x4 = Conv2D(32, (3,3), activation='relu', padding='same')(x4)
x4 = Conv2D(32, (3,3), activation='relu', padding='same')(x4)

#Multi-scale model (Du et al., 2018)
layer_1 = Conv2D(16, (5,5), activation='relu', padding='same')(input_img)
x1m = Conv2D(8, (5,5), activation='relu', padding='same')(layer_1)
x1m = Conv2D(8, (5,5), activation='relu', padding='same')(x1m)

layer_2 = Conv2D(16, (9,9), activation='relu', padding='same')(input_img)
x2m = Conv2D(8, (9,9), activation='relu', padding='same')(layer_2)
x2m = Conv2D(8, (9,9), activation='relu', padding='same')(x2m)

layer_3 = Conv2D(16, (13,13), activation='relu', padding='same')(input_img)
x3m = Conv2D(8, (13,13), activation='relu', padding='same')(layer_3)
x3m = Conv2D(8, (13,13), activation='relu', padding='same')(x3m)

x_add = Concatenate()([x1m, x2m, x3m, input_img])
x4m = Conv2D(8, (7,7), activation='relu', padding='same')(x_add)
x4m = Conv2D(3, (5,5), activation='relu', padding='same')(x4m)

x_final = Concatenate()([x4, x4m])
x_final = Conv2D(dim, (3,3), padding='same')(x_final)
autoencoder = Model(input_img, x_final)
# Using MSE as performance indicator
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# Define file output names
fdir = "Models/"
if not os.path.isdir(fdir):
    os.makedirs(fdir)

# Prepare the file names for parameter summary, model and training history

#fmodel = "Model_Inp_{}_{}_Kernel_{}_{}_{{epoch:02d}}.hdf5".format(var, pooling, kernel, today)
fmodel = "Model_Inp_{}_{}_Kernel_{}_{}.hdf5".format(var, pooling, kernel, today)
fmodel = os.path.join(fdir, fmodel)

fhist = "History_Inp_{}_{}_Kernel_{}_{}.csv".format(var, pooling, kernel, today)
fhist = os.path.join(fdir, fhist)

fsum = "Summary_Inp_{}_{}_Kernel_{}_{}.txt".format(var, pooling, kernel, today)
fsum = os.path.join(fdir, fsum)

# Check if filename already exists and if it does uniquify it
if os.path.isfile(fmodel):
    fmodel, fhist, fsum = uniquify([fmodel, fhist, fsum])

In [ ]:
# Define Callbacks
model_cb = ModelCheckpoint(fmodel, monitor='val_loss',
                           save_best_only=True, verbose=1)
early_cb = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
cb = [model_cb, early_cb]

# Train Neural Network and measure how long training takes
t0 = time()
history = autoencoder.fit(X_train, y_train, epochs=5000, batch_size=batch_size,
                          verbose=1, callbacks=cb, shuffle=shuffle,
                          validation_data=(X_valid, y_valid))
t0 = time()-t0

In [ ]:
# Before loading and evaluating test data free memory
del X_train, y_train, X_valid, y_valid

# Evaluate with a year of data
X_test = np.zeros((len(serial_test), grid[0], grid[1], dim))
y_test = np.zeros((len(serial_test), grid[0],grid[1], dim))
# Iterate through all the datasets and save them in their arrays
for i, s in enumerate(serial_test):

    # Load reference data
    df = pd.read_csv(fname_HR.format(s), header=None, delim_whitespace=False)
    y_test[i,:,:,0] = df.values

    # Load input data
    df = pd.read_csv(fname_LR.format(s), header=None, delim_whitespace=False)
    X_test[i,:,:,0] = df.values

X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

ev = autoencoder.evaluate(X_test, y_test)

# Save Model History
df_results = pd.DataFrame(history.history)
df_results['epoch'] = history.epoch
df_results.to_csv(fhist, index=False)

min_val_loss = min(history.history["val_loss"])
# Save Model Summary
with open(fsum, "w") as f:
    summary = summary.format(today, kernel, pooling, var, train_size, shuffle, patience,
                             batch_size, t0, random_state_train, random_state_test,
                             sample_start, sample_end, min_val_loss, ev, n_augm)
    f.write(summary)